In [ ]:
import serial
import time
import numpy as np
import tdwf
import matplotlib.pyplot as plt
import scipy.optimize as so
import time
import psutil, os, sys


# -[Pre-Processing work]-------------------------------------------
p = psutil.Process(os.getpid())
os_used = sys.platform
if os_used == "win32":  # Windows (either 32-bit or 64-bit)
    p.nice(psutil.REALTIME_PRIORITY_CLASS)
elif os_used == "linux":  # linux
    p.nice(psutil.IOPRIO_HIGH)
else:  # MAC OS X or other
    p.nice(-20)  



# -[Parameters]-------------------------------------------

# 1. hardware parameters
port = 'COM7'  # serial port for the PR59
R = 1e4 # shunt resistance [Ohm]
expected_Vj = 0.7 # expected junction Voltage [V]

# 2. Measurement parameters
nT = 10 # number of temperature points to measure
T_range = [10, 70] # temperature range [°C]
temperatures = np.linspace(T_range[0], T_range[1], nT) # temperatures at which to measure
flag_return_T = True # if true the measurement are done both increasing and lowering the temperature

nv = 10 # number of voltage points to measure
V_range = [-5, 5] # voltage range [V]
offsets = np.linspace(V_range[0], V_range[1], nv) # voltages at which to measure
flag_return_V = True # if true the IV charatestics is measured in both directions

# 3. Plot Parameters
flag_show = True # Plotting the data while measuring
flag_save = True # Saving the data in a file, not sure why you should ever turn it off but ehy, you can

'''
# -[Serial Connection initialization and startup]-------------------------------------------

# 1. Connection setup
ser = serial.Serial()
ser.port = port
ser.baudrate = 115200   
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

# 2. Communcation function definition
# sending and receiving data to the device
def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n\r'
        ser.write(message.encode('utf-8'))
        res = ser.readline().decode()
    if flagR:
        res = ser.readline().decode()
        return res
    
# 3. Testing the connection
swap = query("$V?")
print(swap)
time.sleep(1)
'''

# -[PID Configuration]-------------------------------------------
# technically should be already done in device, leave here just in case


# -[AD2 Connection initialization and startup]-------------------------------------------

# 1. Connection
ad2 = tdwf.AD2()

# 2. Function generator setup
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.func = tdwf.funcDC
wavegen.w1.start()

# 3. Analog input setup
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 1e6
scope.npt = 8192
scope.ch1.range = 10
scope.ch2.range = 10


# -[Plot initialization]-------------------------------------------
fig, [[ax1, ax3, ax5], [ax2, ax4a, ax6]] = plt.subplots(2, 3, figsize=(10, 8), dpi=100)
ax4b = ax4a.twinx()
fig.canvas.manager.set_window_title('IV Measurement - T dependence')
fig.suptitle('IV Measurement - T dependence', fontsize=18)




Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #1


Text(0.5, 0.98, 'IV Measurement - T dependence')

: 

In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 

flag_testing = True
temp = []



# -[Temperature Measurement Function]-------------------------------------------
def measure_temp():
    if flag_testing:
        data = np.sin(len(temp)/10) * .10 + np.sin(len(temp)/2) * .05 + temp_desired# test data with obvious pattern
    else:
        data = query("$R100?")
    
    temp.append(data)
    return data


# -[IV Measurement Function]-------------------------------------------
def measureIV(offsets):
    flag_first = True
    Vd = np.full((nv, 2), np.nan) # diode voltage data
    Vr = np.full((nv, 2), np.nan) # resistor voltage data
    Id = np.full((nv, 2), np.nan) # current data

    for ar in range(2 if flag_return_V else 1):
        if ar == 1:
            offsets = np.flip(offsets)
        for i in range(nv):
            # 1. Set voltage
            wavegen.w1.offs = offsets[i]


            # 2. Measure temperature
            measure_temp()

            # 2. Voltage sampling and automatic fitting
            scope.sample()

            fitfuncDC = lambda x, o: np.array( [o for x in x] )
            pp1, cov1 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch1.vals, p0=[wavegen.w1.offs])
            pp2, cov2 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch2.vals, p0=[wavegen.w1.offs])

            Vd[i, ar] = pp1[0] # diode voltage
            Vr[i, ar] = pp2[0] # resistor voltage
            Id[i, ar] = pp2[0] / R

            if flag_show:
                if flag_first:
                    flag_first = False
                    hp1_true, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp1_sym, = ax1.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp1), "--", label="Fit Ch1", color="#00e7e7")
                    ax1.grid(linestyle='-.')
                    ax1.set_ylabel("Vd [V]", fontsize=15)
                    ax1.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])
                    ax1.set_title(f"Time plots", fontsize=15)


                    hp2_true, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    hp2_sym, = ax2.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp2), "--", label="Fit Ch2", color="#e500a7")
                    ax2.grid(linestyle='-.')
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Vr [A]", fontsize=15)
                    ax2.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])


                    hp3A, = ax3.plot(Vd[:, 0], Id[:, 0], ".", markerfacecolor = "none", label="Go Run", color="tab:purple")
                    if flag_return_V:
                        hp3R, = ax3.plot(Vd[:, 1], Id[:, 1], "v",  markerfacecolor = "none", label="Return Run", color="tab:cyan")
                    
                    ax3.grid(linestyle='-.')
                    ax3.set_xlabel("Vd [V]", fontsize=15)
                    ax3.set_ylabel("Id = Vr/R [A]", fontsize=15)
                    ax3.set_xlim([min(offsets)-0.5, expected_Vj])
                    ax3.set_ylim([min(offsets)/R-0.2/R, max(offsets)/R])
                    ax3.legend(loc='upper left')
                    ax3.set_title(f"I-V Charateristic", fontsize=15)


                    hp4VA, = ax4a.plot(offsets, Vd[:, 0], ".", markerfacecolor = "none", label="Vd Go", color="tab:orange")
                    if flag_return_V:
                        hp4VR, = ax4a.plot(offsets, Vd[:, 1], "v",  markerfacecolor = "none", label="Vd Return", color="tab:orange")                
                    ax4a.grid(linestyle='-.')
                    ax4a.set_xlabel("Vcc [V]", fontsize=15)
                    ax4a.set_ylabel("Vd [V]", fontsize=15)
                    ax4a.tick_params(axis='y', labelcolor="tab:orange")
                    ax4a.set_xlim([min(offsets)-0.5, 1.2*max(offsets)])
                    ax4a.set_ylim([min(offsets)-0.5, 1.2*expected_Vj])

                    
                    hp4IA, = ax4b.plot(offsets, Vr[:, 0], ".", markerfacecolor = "none", label="Id Go", color="tab:blue")
                    if flag_return_V:
                        hp4IR, = ax4b.plot(offsets, Vr[:, 1], "v",  markerfacecolor = "none", label="Id Return", color="tab:blue")
                    ax4b.set_ylabel("Id = Vr/R [A]", fontsize=15)
                    ax4b.tick_params(axis='y', labelcolor="tab:blue")
                    ax4b.set_ylim([min(offsets)/R-0.2/R, max(offsets)/R])



                    plt.tight_layout()
                    plt.show(block=False)    
            else:
                hp1_true.set_xdata(1000*scope.time.vals)
                hp1_true.set_ydata(scope.ch1.vals)
                hp1_sym.set_xdata(1000*scope.time.vals)
                hp1_sym.set_ydata(fitfuncDC(scope.time.vals, *pp1))
                hp2_true.set_xdata(1000*scope.time.vals)
                hp2_true.set_ydata(scope.ch2.vals)
                hp2_sym.set_xdata(1000*scope.time.vals)
                hp2_sym.set_ydata(fitfuncDC(scope.time.vals, *pp2))
                if ar==0:
                    hp3A.set_xdata(Vd[:, 0])
                    hp3A.set_ydata(Id[:, 0])

                    hp4VA.set_xdata(offsets[:])
                    hp4VA.set_ydata(Vd[:, 0])
                    hp4IA.set_xdata(offsets[:])
                    hp4IA.set_ydata(Id[:, 0])
                else:
                    hp3R.set_xdata(Vd[:, 1])
                    hp3R.set_ydata(Id[:, 1])

                    hp4VR.set_xdata(offsets[:])
                    hp4VR.set_ydata(Vd[:, 1])
                    hp4IR.set_xdata(offsets[:])
                    hp4IR.set_ydata(Id[:, 1])

    return Vd, Id


# -[Temperature Plots Update]---------------------------------------------
def update_Tplots():
    ax5.clear()
    ax6.clear()

    fig.suptitle("User interaction: x|c|space|escape|e(DONT BE SILLY, GOOSE)")

    ax5.set_title("Full data acquisition")
    ax5.plot(np.arange(0, len(temp)), temp, "-", label="Temp Data", color="tab:blue")
    ax5.set_xlabel("Sample", fontsize=15)
    ax5.set_ylabel("Temperature [°C]", fontsize=15)
    ax5.legend(loc='upper right')
    ax5.grid(linestyle='-.')

    secondGraphMax = 200
    if(len(temp) < 200):
        secondGraphMax = len(temp)

    ax6.set_title("Latest 200 samples")
    ax6.plot(np.arange(len(temp)-secondGraphMax, len(temp)), temp[-secondGraphMax:], "-", label="Latest Temp Data", color="tab:orange")
    ax6.set_xlabel("Sample", fontsize=15)
    ax6.set_ylabel("Temperature [°C]", fontsize=15)
    ax6.annotate(f"Latest Value [°C]: {temp[-1]:.2f}", (0.4,0.96), xycoords='axes fraction', fontsize=12)
    ax6.legend(loc='upper right')
    ax6.grid(linestyle='-.')

    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.show(block=False)






# -[Main Loop]-------------------------------------------

for ar in range(2 if flag_return_T else 1):
    if ar == 1:
        temperatures = np.flip(temperatures)

    for temp_desired in temperatures:
        # 1. Set temperature
        if not flag_testing:
            query(f"$R0={temp}")

        while True:
            # 2. Wait for temperature to stabilize
            measure_temp()


            if(len(temp) >= 200):
                if abs(max(temp[-200:]) - temp_desired) < 0.05*temp_desired and abs(min(temp[-200:]) - temp_desired) < 0.05*temp_desired:
                    break
            
            update_Tplots()
        
        ax1.clear()
        ax2.clear()
        ax3.clear()
        ax4a.clear()
        ax4b.clear()
        fig.suptitle(f"IV Measurement - T = {temp_desired:.2f} °C", fontsize=18)        
        measureIV(offsets)







'''
# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq

    if event.key == 'x':  # ⇒ export su file
        print("NOT YET IMPLEMENTED")
        filename = input("Esporta dati su file: ")
        dataposition = '../../../Data/'
        data = np.column_stack((np.arange(0, len(temp)), temp))
        info = "# Data acquired from temp PID controller on "+time.strftime("%Y-%m-%d %H:%M:%S")+"\n"
        info += "# Data format: [Sample, Temperature [°C]]\n"
        np.savetxt(dataposition+filename+'.txt', data, delimiter='\t', header=info)

    if event.key == 'c':  # ⇒ send command to scope
        message = input(">> ")
        print(query(message))

    if event.key == ' ':  # ⇒ run/pausa misura 
        flag_acq = not flag_acq

    if event.key == 'escape':  # ⇒ esci dalla misura
        flag_run = False


###    HIGHLY DANGEROUS - DO NOT BE A SILLY GOOSE
    if event.key == 'e':  # ⇒ erase all previous data
        temp.clear()

        print("Erased all data!")
        print("You are a silly goose!")
'''







KeyboardInterrupt: 

In [ ]:
ser.close()

In [3]:
ad2.close()

Dispositivo disconnesso.
